In [1]:
import datetime
import numpy as np
import tensorflow as tf
import sys
from pandas import read_csv
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.models import model_from_json
import matplotlib.pyplot as plt
from IPython.display import clear_output
np.random.seed(123)  # for reproducibility

# Extra part for memory management

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

Using TensorFlow backend.


In [2]:
def Make_LSTM_SSN(train_index, val_index, test_index, Output_Type, state_condition='Stateless', shuffle='True', 
                     pred_hour=156, LSTM_hour=156, num_unit=240, num_epoch=1200, optimizer='Adam', loss_ftn='mean_squared_error'):
    
    if Output_Type is "OneTime" or Output_Type is "StepUp":
        print("Output : ", Output_Type)
    else:
        print, 'You entered wrong Output_Type! Have to enter "StepUp" or "OneTime"'
        sys.exit()        

  
    len_PRED = int(pred_hour)
    len_LSTM = int(LSTM_hour)
    
    file_condition = str(LSTM_hour)+'LSTM' + str(LSTM_hour)+'Prediction'
    
    if len_PRED <= 0:
        print("You entered wrong predction hour or peak hour")
        sys.eixt()
    if len_LSTM <= 0:
        print("You entered wrong LSTM hour or peak hour")
        sys.eixt()
        
        
    if Output_Type is "OneTime":
        allDataSet = read_csv('C:\\Users\\YKW\\GoogleDrive\\Study USB BackUp\\SSN\\SSN_LSTM_csv\\SNN_M_'+file_condition+'.csv', sep=',')
    else:
        if Output_Type is "StepUp":
            allDataSet = read_csv('C:\\Users\\YKW\\GoogleDrive\\Study USB BackUp\\SSN\\SSN_LSTM_csv\\SSN_M_'+str(LSTM_hour)+'LSTM_1Prediction.csv', sep=',')
    
    allDataSet_size = len(allDataSet)
                                  
    
    print("allDataSet: ", allDataSet.shape)
    print("allDataSet_size: ", allDataSet_size)
    print("allDataSet Type: ", type(allDataSet))
    print("allDataSet_size Type: ", type(allDataSet_size))
    
    LSTM_count = np.arange(len_LSTM)+1
    LSTM_count = LSTM_count* (-1)
    LSTM_count = LSTM_count +len_LSTM+1
    LSTM_title = []
    for bi in range(len(LSTM_count)):
        LSTM_title.append('b'+str(int(LSTM_count[bi])))
        
    pred_count = np.arange(len_PRED)
    pred_title = []
    for ai in range(len(pred_count)):
        pred_title.append('a'+str(int(pred_count[ai])))
        
#    print(LSTM_title)
#    sys.exit()
    
    LSTM_title1 = LSTM_title[0:30]
    LSTM_title2 = LSTM_title[30:60]
    LSTM_title3 = LSTM_title[60:90]
    LSTM_title4 = LSTM_title[90:120]
    LSTM_title5 = LSTM_title[120:133]
    
#    print(LSTM_title1)
#    sys.exit()

    
    flux1 = allDataSet[LSTM_title1].values
    flux2 = allDataSet[LSTM_title2].values   
    flux3 = allDataSet[LSTM_title3].values   
    flux4 = allDataSet[LSTM_title4].values
    flux5 = allDataSet[LSTM_title5].values
    
    sys.exit()

   
    now_flux = allDataSet['a0'].values
    now_Year = allDataSet['Year'].values
    now_Mon = allDataSet['Mon'].values
     
    d_flux = np.array([flux])    
    d_now_flux = np.array([now_flux])
    d_now_Year = np.array([now_Year])
    d_now_Mon = np.array([now_Mon])   
  
    d_flux = np.array(d_flux).reshape(allDataSet_size, len_LSTM,1)    
    d_now_flux = np.array([d_now_flux]).reshape(allDataSet_size, 1)
    d_now_Year = np.array([d_now_Year]).reshape(allDataSet_size, 1)
    d_now_Mon = np.array([d_now_Mon]).reshape(allDataSet_size, 1)
                                  
    if Output_way is "Onetime":
        pred_flux = allDataSet[pred_title].values
        d_pred_flux = np.array([pred_flux])
        d_pred_flux = np.array(d_pred_flux).reshape(allDataSet_size, len_PRED)
                                  
    
                                  
    def M_LSTM_Stateful(shapes):
        model = Sequential()
        model.add(LSTM(num_unit,batch_input_shape=shapes, stateful=True))
        if Output_type is "StepUp":
            model.add(Dense(1))
        else:
            if Output_type is "OneTime":
                model.add(Dense(len_PRED))
        model.summary()
        return model
            
    def M_LSTM_Stateless(shapes):
        model = Sequential()
        model.add(LSTM(num_unit,input_shape=shapes, stateful=False))  # model.add(LSTM(num_unit,input_shape=shapes, return_sequences=True, stateful=False))
        if Output_type is "StepUp":
            model.add(Dense(1))
        else:
            if Output_type is "OneTime":
                model.add(Dense(len_PRED))
        model.summary()
        return model


    class PlotLosses(keras.callbacks.Callback):
        def on_train_begin(self, logs={}):
            self.i = 0
            self.x = []
            self.losses = []
            self.val_losses = []

            self.fig = plt.figure()

            self.logs = []

        def on_epoch_end(self, epoch, logs={}):

            self.logs.append(logs)
            self.x.append(self.i)
            self.losses.append(logs.get('loss'))
            self.val_losses.append(logs.get('val_loss'))
            self.i += 1
            clear_output(wait=True)
            plt.plot(self.x, self.losses, label="loss")
            plt.plot(self.x, self.val_losses, label="val_loss")
            plt.legend()
            plt.show();
            

    if shuffle is 'True':
        if state_conditon is "Stateful":
            print('You can not set shuffle="True" while state_condition is "Stateful"!')
            sys.exit()
        weightfileName = 'C:\\Users\\YKW\\GoogleDrive\\Study USB BackUp\\SSN\\SSN_LSTM_WEIGHT\\Layer1_shuffled_'+state_condition+'_'+Output_Way+'_LSTM_GOES15_pgt10_'+file_condition+'_ep'+str(num_epoch)+'_'+str(num_unit)+'units_WEIGHT'
        testfileName = 'C:\\Users\\YKW\\GoogleDrive\\Study USB BackUp\\SSN\\SSN_LSTM_RESULT\\Layer1_shuffled_'+state_condition+'_'+Output_Way+'_TEST_'+file_condition+'_Proton_LSTM.csv'
    else:
        if shuffle is 'False':
            weightfileName = 'C:\\Users\\YKW\\GoogleDrive\\Study USB BackUp\\SSN\\SSN_LSTM_WEIGHT\\Layer1_'+state_condition+'_'+Output_Way+'_LSTM_GOES15_pgt10_'+file_condition+'_ep'+str(num_epoch)+'_'+str(num_unit)+'units_WEIGHT'
            testfileName = 'C:\\Users\\YKW\\GoogleDrive\\Study USB BackUp\\SSN\\SSN_LSTM_RESULT\\Layer1_'+state_condition+'_'+Output_Way+'_TEST_'+file_condition+'_Proton_LSTM.csv'
            

    plot_losses = PlotLosses()
    
    if state_condition is 'Stateful':
        inputFluxShape = (1,len_LSTM, 1)
        model = M_LSTM_Stateful(inputFluxShape)

    else:
        if state_condition is 'Stateless':
            inputFluxShape = (len_LSTM,1)  
            model = M_LSTM_Stateless(inputFluxShape)
        else:
            print("You enterd wrong state_condition!")
            sys.exit()
 

    val_index  = train_index + val_index
    test_index = val_index + test_index
    
    trainX = d_flux[:train_index]    
    trainYear = d_now_Year[:train_index]
    trainMon = d_now_Mon[:train_index]
    
    valX = d_flux[train_index:val_index]    
    valYear = d_now_Year[train_index:val_index]
    valMon = d_now_Mon[train_index:val_index]
    
#    testX = d_flux[val_index:test_index]    
#    testYear = d_now_Year[val_index:test_index]
#    testMon = d_now_Mon[val_index:test_index]
                  
    testX = valX
    testYear = valYear
    testMon = valMon
    
    if Output_type is "StepUp":
            trainY = d_now_flux[:train_index]
            valY = d_now_flux[train_index:val_index]
#            testY = d_now_flux[val_index:test_index]
            testY = valY
    else:
        if Output_type is "OneTime":
            trainY = d_pred_flux[:train_index]
            valY = d_pred_flux[train_index:val_index]
#           testY = d_pred_flux[val_index:test_index]
            testY = valY

    
    model.compile(loss=loss_ftn, optimizer=optimizer, metrics=['acc'])
    
    
    if state_condition is 'Stateful':    
        for epoch_idx in range(num_epoch):
            if shuffle is 'True':
                model.fit(trainX, trainY, epochs=1, batch_size=1, verbose=2, shuffle=True, validation_data=(valX, valY))
                model.reset_states()
            else:
                if shuffle is 'False':
                    model.fit(trainX, trainY, epochs=1, batch_size=1, verbose=2, shuffle=False, validation_data=(valX, valY))
                    model.reset_states()
            print(epoch_idx)
    else:
        if state_condition is 'Stateless': 
            if shuffle is 'True':
                model.fit(trainX, trainY, epochs=num_epoch, batch_size=1400, verbose=2, shuffle=True, validation_data=(valX, valY))
            else:
                if shuffle is 'False':
                    model.fit(trainX, trainY, epochs=num_epoch, batch_size=1400, verbose=2, shuffle=False, validation_data=(valX, valY))

        
    model_json = model.to_json()
    with open(weightfileName + ".json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights(weightfileName + ".h5")
    print("Saved model to disk")

    json_file = open(weightfileName + '.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)

    loaded_model.load_weights(weightfileName + ".h5")
    print("Loaded model from disk")
    loaded_model.compile(loss=loss_ftn, optimizer=optimizer, metrics=['acc'])
    

    f = open(testfileName, 'w')
    f.write(Ouput_way+'_'+state_condition+'shuffle='+shuffle+'_'+str(num_epoch)+'epochs_'+str(num_unit)+'units_'+optimizer + ' optimizer_' + loss_ftn+' loss function')
    f.write('\n')
    f.write('Year' + ','+ 'Month' + ',' + 'Ground')
    for i in range(len_PRED):
        f.write(','+ 'a'+str(i))
    f.write('\n')
    
    if Ouput_way is "StepUp":
        for idx2 in range(0, len(testX)):        

            pred = []
            pred_in = testX[idx2:idx2+1]  #    (1, 1, 96)            

            for pred_n in range(len_PRED):
                resultY = model.predict(pred_in)
                pred.append(resultY)

                pred_in = np.append(pred_in,resultY)
                pred_in = pred_in[1:]
                pred_in = pred_in.reshape(1, len_LSTM, 1)

            date = str(Year) + ',' + str(Mon)
            f.write(date)
            f.write(',' + str(NowY[0][0]))

            for nwr in range(len_PRED):
                f.write(',' + str(pred[nwr][0][0]))   

            f.write('\n')
            pred_in = []
            resultY = []
            print(idx2)

        f.close()
                  
    
    if Output_way is "OneTime":
        for idx2 in range(0, len(testX)):

            NowY = testNowY[idx2:idx2+1]

            Year = testYear[idx2][0]
            Mon = testMon[idx2][0]

            pred = []
            pred_in = testX[idx2:idx2+1]      

            resultY = model.predict(pred_in)

            date = str(Year) + ',' + str(Mon)
            f.write(date)
            f.write(',' + str(NowY[0][0]))

            for nwr in range(len_PRED):
                f.write(',' + str(resultY[0][nwr]))   

            f.write('\n')
            pred_in = []
            resultY = []
            print(idx2)

        f.close()
            

###################################################################################################

In [3]:
Make_LSTM_SSN(2856, 144, 1, "StepUp", state_condition='Stateful', shuffle='False', 
              pred_hour=132, LSTM_hour=132, num_unit=240, num_epoch=1200, optimizer='Adam', loss_ftn='mean_squared_error')
Make_LSTM_SSN(2856, 144, 1, "StepUp", state_condition='Stateful', shuffle='False', 
              pred_hour=132, LSTM_hour=264, num_unit=240, num_epoch=1200, optimizer='Adam', loss_ftn='mean_squared_error')
    
Make_LSTM_SSN(2856, 144, 1, "StepUp", state_condition='Stateful', shuffle='False', 
              pred_hour=132, LSTM_hour=132, num_unit=240, num_epoch=600, optimizer='Adam', loss_ftn='mean_squared_error')
Make_LSTM_SSN(2856, 144, 1, "StepUp", state_condition='Stateful', shuffle='False', 
              pred_hour=132, LSTM_hour=132, num_unit=480, num_epoch=1200, optimizer='Adam', loss_ftn='mean_squared_error')
Make_LSTM_SSN(2856, 144, 1, "StepUp", state_condition='Stateful', shuffle='False', 
              pred_hour=132, LSTM_hour=132, num_unit=480, num_epoch=600, optimizer='Adam', loss_ftn='mean_squared_error')

## 1998~2009 Test

Output :  StepUp
allDataSet:  (3102, 135)
allDataSet_size:  3102
allDataSet Type:  <class 'pandas.core.frame.DataFrame'>
allDataSet_size Type:  <class 'int'>


KeyError: "['b99' 'b98' 'b97' 'b96' 'b95' 'b94' 'b93' 'b92' 'b91' 'b90' 'b89' 'b88'\n 'b87' 'b86' 'b85' 'b84' 'b83' 'b82' 'b81' 'b80' 'b79' 'b78' 'b77' 'b76'\n 'b75' 'b74' 'b73'] not in index"